In [1]:
import numpy as np
import pandas as pd

In [2]:
from bs4 import BeautifulSoup

In [3]:
import requests
import time

In [4]:
url = 'https://www.chinatimes.com/search/韓國瑜?chdtv&page'

In [5]:
res = requests.get(url)
# print(res.text) #查看抓到的原始碼的長相 (因為很長，這邊就不列出來了)

In [6]:
soup = BeautifulSoup(res.text, 'lxml')

In [7]:
count = soup.find('span',{'class':'search-result-count'}).get_text().replace(',','') #看有幾筆資料
print(count)

18035


In [10]:
df = pd.DataFrame({'title':[0] ,'context': [0],'url':[0] }) #創一個表格
print(df)

   title  context  url
0      0        0    0


In [11]:
for i in range(0,10): #判斷總頁數有多少可以用：int(np.ceil(int(count)/20)) (np.ceil是無條件進位)，先判斷頁數有幾頁，是否需要換頁 
    #但韓總實在太多筆了，我們先爬前十頁就好
    url = 'https://www.chinatimes.com/search/韓國瑜?chdtv&page='+str(i+1)
    res = requests.get(url) #取得網頁資料
    soup = BeautifulSoup(res.text, 'lxml') #將網頁資料存到BeautifulSoup4可以讀取的格式
    print(i)
    for context in soup.find_all('div',{'class':'col'}): #從網頁的程式碼中找需要的資料。
        df1 = pd.DataFrame({'title':[context.find('h3',{'class':'title'}).get_text()],
                            'context':[context.find('p',{'class':'intro'}).get_text()],
                            'url':[context.find('a').get('href')]})
        df = pd.concat([df,df1],axis=0)
#         time.sleep(2) #設定停頓時間

0
1
2
3
4
5
6
7
8
9


In [12]:
df.reset_index(inplace=True) #重設index

In [13]:
df.drop([0],axis=0,inplace=True) #把不要的資料欄位移除
df.drop(['index'],axis=1,inplace=True)

In [14]:
df

,title,context,url
1,狂！韓國瑜一句話嘴爆小英 網：太中肯了,昨日國民黨總統候選人韓國瑜與總統蔡英文兩人在嘉義尬場造勢，兩人會場相隔一公里，較勁意味濃厚，...,https://www.chinatimes.com/realtimenews/201911...
2,誇張！親綠民調調查 韓國瑜在這點竟大輸蔡英文？,《台灣民意基金會》今日公布最新民調結果，總統蔡英文在支持度上再度以55.2%力壓國民黨總統候...,https://www.chinatimes.com/realtimenews/201911...
3,韓國瑜吮珍奶吐一句話 網驚喊母湯！搶著幫他搖,國民黨總統候選人韓國瑜24日行程滿檔，但仍不忘在社群媒體與網友互動，關心粉絲之餘不忘展露韓式...,https://www.chinatimes.com/realtimenews/201911...
4,民眾瘋搶韓簽名T恤 最高10萬決標,國民黨總統參選人韓國瑜24日參加嘉義市萌寵音樂嘉年華會，與市長黃敏惠、國民黨立委參選人傅大...,https://www.chinatimes.com/newspapers/20191125...
5,韓酸綠高高在上 心沒放人民身上,國民黨總統參選人韓國瑜24日在南投與立委許淑華聯合造勢，因小英昨日也在南投，藍營輸人不輸陣...,https://www.chinatimes.com/newspapers/20191125...
...,...,...,...
196,小英買地買房大手筆 1450花錢不手軟,當綠營猛烈攻擊韓國瑜房產，甚至連十年前只是住過的房子都被翻出來，相對之下蔡英文炒房炒地卻都沒...,https://www.chinatimes.com/realtimenews/201911...
197,傅崐萁參選後果 名嘴直言：非常可怕,力挺國民黨總統候選人韓國瑜的無黨籍前花蓮縣長傅崐萁，確定參選花蓮立委，為選情投下震撼彈！對此...,https://www.chinatimes.com/realtimenews/201911...
198,24歲站上第一線！年輕正妹吐挺韓心聲,國民黨總統候選人韓國瑜自從投入大選後，就遭到民進黨窮追猛打的抹黑成草包。導致他在年輕族群支...,https://www.chinatimes.com/realtimenews/201911...
199,羅智強：民進黨不分區名單 更爛,國民黨台北市議員羅智強21日接受「高雄林小姐」訪問時表示，民進黨不分區立委名單，比國民黨更爛...,https://www.chinatimes.com/realtimenews/201911...


In [15]:
df.to_excel('中時電子報韓國瑜爬蟲結果.xlsx') #輸出資料